In [ ]:
import pandas as pd
import yfinance as yf
from datatime import datetime
from ib_insync import * util.startLoop()

df = pd.read_csv("DJI_Const.csv", header = [0,1], index_col = 0, parse_dates = [0])
print(df)

symbols = df.Close.columns.to_list()
print(symbols)

symbols.remove("DJI")
len(symbols)

Get most recent prices and performance

In [ ]:
cprice = yf.Ticker("AAPL").get_info()["regualerMarketPrice"]
print(cprice)

last_close = yf.Ticker("AAPL").get_info()["regularMarketPreviousClose"]
print(last_close)

perf = cprice / last_close - 1
print(perf)

perf = pd.Series(dtype = float)
print(perf)

count = 1
for symbol in symbols:
  try:
    info = yf.Ticker(ticker = symbol).get_info()
    prc_chg = info["regularMarketPrice"] / info["regularMarketPreviousClose"] - 1
    perf.loc[symbol] = prc_chg
    print(count, end = "\r")
    count += 1
  except Exception as e:
    print("{} not found".format(symbol))
print("Download complete")

perf.sort_values(inplace = True)
print(perf)

perf.index.name = "symbol"
print(perf)

Determine Target Positions

In [ ]:
buy_stocks = 3 # buy the 3 worst performing stocks
sell_stocks = 3 # short sell the 3 best performing stocks
shares = 1 # one share per stock

perf.iloc[:buy_stocks] = shares
perf.iloc[-sell_stocks:] = -shares
print(perf)

target = pd.concat([perf.iloc[:buy_stocks]]).to_frame().reset_index()
target.columns = ["symbol", "position"]
print(target)

Identify Current Positions

In [ ]:
ib = IB()
ib.connect()
pos = ib.postions()
print(pos)

df = util.dif(pos)
print(df)

df["symbol"] = df.contract.apply(lambda x: x.symbol)
df["conID"] = df.contract.apply(lambda x: x.conId)
print(df)
print(df.info())

if df is not None:
  df["symbol"] = df.contract.apply(lambda x: x.symbol)
  df["conID"] = df.contract.apply(lambda x: x.conId)  
else:
  df = pd.DataFrame(columns = ["symbol", "position"])
print(df)

In [ ]:
trades = pd.merge(target, df[["symbol", "position"]], "outer", on = "symbol", suffixes = ["_t", "_a"])
print(trades)

trades.fillna(0, inplace = True)
print(trades)

trades["trades"] = trades.position_t - trades.position_a
print(trades)

trades = trades[trades.trades != 0].set_index("symbol").copy()
print(trades)

Execute Trades

In [ ]:
for symbol in trades.index:
  to_trade = trades.loc[symbol, "trades"]
  if to_trade > 0:
    side = "BUY"
  elif to_trade < 0:
    side = "SELL"
  contract = Stock(symbol, "SMART", "USD")
  cds = ib.reqContractDetials(contract)
  if len(cds) == 0:
    print("No Contract for {} found").format(symbol))
  elif len(cds) == 1:
    contract = cds[0].contract
    order = MarketOrder(side, abs(to_trade))
    trade = ib.placeOrder(contract, order)
    while not trade.isDone():
      ib.waitOnUpdate()
    if trade.orderStatus.status == "Filled":
      print("{} {} @ {}". format(side, symbol, trade.orderStatus.vgFillPrice))
    else:
      print("{} {} failed.".format(side, symbol))
  else:
    contract = cds[0].contract
    print("Multiple Contracts for {} found.".format(symbol))
    order = MarketOrder(side, abs(to_trade))
    trade = ib.placeOrder(contract, order)
    while not trade.isDone():
      ib.waitOnUpdate()
    if trade.orderStatus.status == "Filled":
      print("{} {} @ {}".format(side, symbol, trade.orderStatus.avgFillPrice))
    else:
      print("{} {} failed.". format(side, symbol))
pos = ib.positions()
df = util.df(pos)
df["symbol"] = df.contract.apply(lambda x: x.symbol)
df["conID"] = df.contract.apply(lambda x: x.conId)
print(df)